In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mcdonalds-store-reviews/McDonald_s_Reviews.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense,Bidirectional,Embedding,Conv1D,GlobalAveragePooling1D,Flatten,GRU,LSTM
from tensorflow.keras.utils import to_categorical

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
path="../input/mcdonalds-store-reviews/McDonald_s_Reviews.csv"

In [4]:
data=pd.read_csv(path,encoding="ISO-8859-1")
data.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33396 entries, 0 to 33395
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   reviewer_id    33396 non-null  int64  
 1   store_name     33396 non-null  object 
 2   category       33396 non-null  object 
 3   store_address  33396 non-null  object 
 4   latitude       32736 non-null  float64
 5   longitude      32736 non-null  float64
 6   rating_count   33396 non-null  object 
 7   review_time    33396 non-null  object 
 8   review         33396 non-null  object 
 9   rating         33396 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 2.5+ MB


In [6]:
data["rating"].unique()

array(['1 star', '4 stars', '5 stars', '2 stars', '3 stars'], dtype=object)

In [7]:
ratings={"1 star":"Terrible","2 stars":"Bad","3 stars":"Neutral","4 stars":"Good","5 stars":"Excellent"}
data.replace(ratings,inplace=True)
data.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,Terrible
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,Good
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,Terrible
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,Excellent
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",Terrible


In [8]:
corpus=[word.lower().split("\n") for word in data["review"]]
labels=data["rating"].values

In [9]:
train_corpus,test_corpus,train_label,test_label=train_test_split(corpus,labels,test_size=0.3,random_state=42)

In [10]:
print(train_corpus[:5],train_label[:5])

[['i went today, 11:15am, asked for the free big mac/maccoin, they said, they were not authorized, 08/02/2018'], ['slowest mcds in the entire state of florida!!! stay away unless you want to waste time in the drive-thur'], ['horrible service fire the b,i,t,c,h'], ['this place really should be shut down. if they can\'t deliver in customer service just close.  i ordered with them and waited for 45 minutes.  i asked them after 10 minutes about my order and they said it\'s coming up.  after 20 minutes i ask again about my order and the supervisor tells me "one at a time, i can\'t skip your order in front of other people".  she then proceeds to continue skipping my order to serve other people.  at 45 minutes i asked again and she finds my order on a table and says to her coworker they never made it.   nobody gave an apology or acknowledged they were in the wrong.  if you can\'t run a restaurant correctly then please just shut down.  there are countless pictures on their reviews of cars pili

In [11]:
tokenizer=Tokenizer(num_words=10000,oov_token='OOV')
tokenizer.fit_on_texts(train_corpus)
train_cor=tokenizer.texts_to_sequences(train_corpus)
test_cor=tokenizer.texts_to_sequences(test_corpus)

padded_train=pad_sequences(train_cor,padding="pre",maxlen=max([len(_) for _ in train_cor]))
padded_test=pad_sequences(test_cor,padding="pre",maxlen=max([len(_) for _ in test_cor]))

In [12]:
print(f"Shape of training data are {padded_train.shape} for the text data and {train_label.shape} for the labels")
print(f"Shape of test data are {padded_test.shape} for the text data and {test_label.shape} for the labels")

Shape of training data are (23377, 20) for the text data and (23377,) for the labels
Shape of test data are (10019, 16) for the text data and (10019,) for the labels


In [13]:
model=Sequential()
